In [1]:
import logging
import re
import typing as t
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
from jinja2 import Template
from openai import OpenAI
from tqdm.auto import tqdm


# Openrouter constants
OPENROUTER_API_KEY = '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'
OPENTROUTER_BASE_URL = 'https://openrouter.ai/api/v1'


# Experiment constants
EXPERIMENT_NAME = 'exp1.0'  # baseline
MODEL_NAME = 'openai/gpt-4o-mini'
# MODEL_NAME = 'openai/gpt-4'
TEMPERATURE = 0.0

RANDOM_SEED = 20250402

N_ARMS = 5
DELTA = 0.2
N_TRIALS = 100


ARM_NAME_TO_IDX = {'blue': 0, 'green': 1, 'red': 2, 'yellow': 3, 'purple': 4}
ARM_IDX_TO_NAME = {v: k for k, v in ARM_NAME_TO_IDX.items()}


DATA_DIR = Path('../data')
EXPERIMENT_DIR = DATA_DIR / EXPERIMENT_NAME
EXPERIMENT_DIR.mkdir(exist_ok=True, parents=True)


def bootstrap() -> None:
    logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s')
    np.random.seed(RANDOM_SEED)


bootstrap()

In [2]:
SYSTEM_PROMPT_TEMPLATE = """\
You are a bandit algorithm in a room with {{n_arms}} buttons labeled blue, green, red, yellow, purple. \
Each button is associated with a Bernoulli distribution with a fixed but unknown mean; the means \
for the buttons could be different. For each button, when you press it, you will get a reward \
that is sampled from the button's associated distribution. You have {{n_trials}} time steps and, on each \
time step, you can choose any button and receive the reward. Your goal is to maximize the total \
reward over the {{n_trials}} time steps.

At each time step, I will show you a summary of your past choices and rewards. Then you must \
make the next choice, which must be exactly one of blue, green, red, yellow, purple. Let's think \
step by step to make sure we make a good choice. You must provide your final answer within the \
tags <Answer>COLOR</Answer> where COLOR is one of blue, green, red, yellow, purple.
"""
SYSTEM_PROMPT_TEMPLATE = Template(SYSTEM_PROMPT_TEMPLATE)


USER_PROMPT_TEMLATE = """\
So far you have played {{n_trials}} times with your past choices and rewards summarized
as follows:
- blue button: pressed {{blue_n_occur}} times {%- if blue_n_occur != 0 %} with average reward \
{{blue_avg_reward}}{% endif %}
- green button: pressed {{green_n_occur}} times {%- if green_n_occur != 0 %} with average reward \
{{green_avg_reward}}{% endif %}
- red button: pressed {{red_n_occur}} times {%- if red_n_occur != 0 %} with average reward \
{{red_avg_reward}}{% endif %}
- yellow button: pressed {{yellow_n_occur}} times {%- if yellow_n_occur != 0 %} with average \
reward {{yellow_avg_reward}}{% endif %}
- purple button: pressed {{purple_n_occur}} times {%- if purple_n_occur != 0 %} with average \
reward {{purple_avg_reward}}{% endif %}

Which button will you choose next? Remember, YOU MUST provide your final answer within the tags \
<Answer>COLOR</Answer> where COLOR is one of blue, green, red, yellow, purple. Let's think step \
by step to make sure we make a good choice.
"""
USER_PROMPT_TEMLATE = Template(USER_PROMPT_TEMLATE)

In [3]:
# Rendering example
example_prompt_inputs = {
    'n_trials': 10,
    'blue_n_occur': 4,
    'blue_avg_reward': 0.4,
    'green_n_occur': 0,
    'green_avg_reward': 0,
    'red_n_occur': 3,
    'red_avg_reward': 0.1,
    'yellow_n_occur': 0,
    'yellow_avg_reward': 0,
    'purple_n_occur': 3,
    'purple_avg_reward': 0.2
}
print(USER_PROMPT_TEMLATE.render(**example_prompt_inputs))

So far you have played 10 times with your past choices and rewards summarized
as follows:
- blue button: pressed 4 times with average reward 0.4
- green button: pressed 0 times
- red button: pressed 3 times with average reward 0.1
- yellow button: pressed 0 times
- purple button: pressed 3 times with average reward 0.2

Which button will you choose next? Remember, YOU MUST provide your final answer within the tags <Answer>COLOR</Answer> where COLOR is one of blue, green, red, yellow, purple. Let's think step by step to make sure we make a good choice.


In [4]:
class MultiArmedBandit:
    def __init__(self, n_arms: int, delta: float) -> None:
        self.n_arms = n_arms
        self.delta = delta

        means = [0.5 + self.delta / 2] + [0.5 - self.delta / 2] * (self.n_arms - 1)
        np.random.shuffle(means)

        self.means = means
        self.best_arm = np.argmax(means)

    def pull(self, arm: int) -> int:
        assert 0 <= arm < self.n_arms, f"Arm {arm} doesn't exist"

        p = self.means[arm]
        reward = np.random.binomial(1, p=p)

        return reward

In [5]:
def create_client() -> OpenAI:
    client = OpenAI(
        base_url=OPENTROUTER_BASE_URL,
        api_key=OPENROUTER_API_KEY,
    )
    return client


def get_prediction(client: OpenAI, system_prompt: str, user_prompt: str, **kwargs) -> str | None:
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            'role': 'user',
            'content': user_prompt
        }
    ]

    prediction = None
    try:
        completion = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=TEMPERATURE,
            **kwargs
        )
        prediction = completion.choices[0].message.content
    except Exception as _:
        logging.warning('Request failed')

    return prediction


def build_prompt_inputs(trial: int, rewards: t.Dict[str, int], occurrences: t.Dict[str, int]) -> t.Dict[str, float]:
    def average(key: str):
        occ = occurrences[key]
        total = rewards[key]
        return total / occ if occ != 0 else 0.0

    prompt_inputs = {
        'n_trials': trial,

        'blue_n_occur': occurrences['blue'],
        'blue_avg_reward': average('blue'),

        'red_n_occur': occurrences['red'],
        'red_avg_reward': average('red'),

        'green_n_occur': occurrences['green'],
        'green_avg_reward': average('green'),

        'yellow_n_occur': occurrences['yellow'],
        'yellow_avg_reward': average('yellow'),

        'purple_n_occur': occurrences['purple'],
        'purple_avg_reward': average('purple'),
    }

    return prompt_inputs


def extract_arm_color(prediction: str) -> str | None:
    names = '|'.join(ARM_NAME_TO_IDX.keys())
    pattern = f'<Answer>({names})</Answer>'
    match = re.search(pattern, prediction)

    color = None
    if match:
        color = match.group(1)

    return color

In [6]:
client = create_client()
bandit = MultiArmedBandit(n_arms=N_ARMS, delta=DELTA)

In [7]:
def run_experiment(client: OpenAI, bandit: MultiArmedBandit) -> None:
    pass

In [8]:
stats = []

rewards = {k: 0 for k in ARM_NAME_TO_IDX.keys()}
occurrences = {k: 0 for k in ARM_NAME_TO_IDX.keys()}

for trial in tqdm(range(N_TRIALS), total=N_TRIALS):
    system_prompt = SYSTEM_PROMPT_TEMPLATE.render({'n_arms': N_ARMS, 'n_trials': N_TRIALS})

    user_prompt_inputs = build_prompt_inputs(trial, rewards, occurrences)
    user_prompt = USER_PROMPT_TEMLATE.render(user_prompt_inputs)
    prediction = get_prediction(client, system_prompt, user_prompt)

    arm_name = extract_arm_color(prediction)
    arm_idx = ARM_NAME_TO_IDX[arm_name]

    reward = bandit.pull(arm_idx)

    occurrences[arm_name] += 1
    rewards[arm_name] += reward

    cumulative_reward = sum(rewards.values())
    cumulative_reward_per_arm = {f'cumulative_reward_{k}': v for k, v in rewards.items()}
    cumulative_occurrences_per_arm = {f'cumulative_occurrence_{k}': v for k, v in occurrences.items()}
    stats.append({
        'trial': trial,
        'arm_name': arm_name,
        'arm_idx': arm_idx,
        'reward': reward,
        'cumulative_reward': cumulative_reward,
        'system_prompt': system_prompt,
        'user_prompt': user_prompt,
        'raw_prediction': prediction,
        'best_arm': ARM_IDX_TO_NAME[bandit.best_arm],
        **cumulative_reward_per_arm,
        **cumulative_occurrences_per_arm
    })

df = pd.DataFrame(stats)

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
model_name = MODEL_NAME.replace('/', '_').replace(':', '_').replace('-', '_')
version = datetime.now().strftime('%Y%m%d%H%M%S')

filename = f'{EXPERIMENT_NAME}_{model_name}_arms-{N_ARMS}_delta-{DELTA}_trials-{N_TRIALS}_v{version}.csv'
filepath = EXPERIMENT_DIR / filename

df.to_csv(filepath, index=False)

NameError: name 'df' is not defined

# Computing evalutation metrics

In [32]:
import pandas as pd
import numpy as np

df = pd.read_csv("exp1.0_openai_gpt_4o_mini_arms-5_delta-0.2_trials-100_v20250402120000.csv")  # Replace with your actual file path
print(df.head())

In [11]:
%ls

exp1.0_openai_gpt_4o_mini_arms-5_delta-0.2_trials-100_v20250402120000.csv  sample_data/


First we will need to combine the different replicate results together into one dataframe

In [34]:
# Combine all the dataframes together
dataframes = []

# Add the file names of the csvs here
#for r in range(10):
#    df = pd.read_csv(f"results_replicate_{r}.csv")
#    df["replicate"] = r
#    dataframes.append(df)

# FOR TESTING
df_1 = df.copy()
df_1["replicate"] = 0
df_2 = df.copy()
df_2["replicate"] = 1
df_3 = df.copy()
df_3["replicate"] = 2
dataframes = [df_1, df_2, df_3]

# Merged dataframe
df_combined = pd.concat(dataframes, ignore_index=True)

# Print result
print(df_combined.head())

   trial arm_name  arm_idx  reward  cumulative_reward  \
0      0     blue        0       0                  0   
1      1    green        1       0                  0   
2      2      red        2       1                  1   
3      3      red        2       0                  1   
4      4      red        2       1                  2   

                                       system_prompt  \
0  You are a bandit algorithm in a room with 5 bu...   
1  You are a bandit algorithm in a room with 5 bu...   
2  You are a bandit algorithm in a room with 5 bu...   
3  You are a bandit algorithm in a room with 5 bu...   
4  You are a bandit algorithm in a room with 5 bu...   

                                         user_prompt  \
0  So far you have played 0 times with your past ...   
1  So far you have played 1 times with your past ...   
2  So far you have played 2 times with your past ...   
3  So far you have played 3 times with your past ...   
4  So far you have played 4 times with y

# Metric 1: SuffFailFreq(T/2) quantifies exploration failures

For an experiment replicate R and round t, let SuffFail(t, R) be a binary variable that is 1 if the best arm is never chosen in rounds [t, T ]. Then let SuffFailFreq(t) := mean({SuffFail(t, R) : replicates R}).
SuffFailFreq(T /2)




In [35]:
df_combined

,trial,arm_name,arm_idx,reward,cumulative_reward,system_prompt,user_prompt,raw_prediction,cumulative_reward_blue,cumulative_reward_green,...,cumulative_occurrence_blue,cumulative_occurrence_green,cumulative_occurrence_red,cumulative_occurrence_yellow,cumulative_occurrence_purple,best_arm,SuffFail,MinFrac,KMinFrac,replicate
0,0,blue,0,0,0,You are a bandit algorithm in a room with 5 bu...,So far you have played 0 times with your past ...,"Since I have not pressed any buttons yet, I ha...",0,0,...,1,0,0,0,0,purple,0,0.000000,0.000000,0
1,1,green,1,0,0,You are a bandit algorithm in a room with 5 bu...,So far you have played 1 times with your past ...,Based on the summary of past choices and rewar...,0,0,...,1,1,0,0,0,purple,0,0.000000,0.000000,0
2,2,red,2,1,1,You are a bandit algorithm in a room with 5 bu...,So far you have played 2 times with your past ...,Based on the summary of past choices and rewar...,0,0,...,1,1,1,0,0,purple,0,0.000000,0.000000,0
3,3,red,2,0,1,You are a bandit algorithm in a room with 5 bu...,So far you have played 3 times with your past ...,Based on the summary of past choices and rewar...,0,0,...,1,1,2,0,0,purple,0,0.000000,0.000000,0
4,4,red,2,1,2,You are a bandit algorithm in a room with 5 bu...,So far you have played 4 times with your past ...,Let's analyze the information we have so far:\...,0,0,...,1,1,3,0,0,purple,0,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,95,purple,4,0,52,You are a bandit algorithm in a room with 5 bu...,So far you have played 95 times with your past...,"To decide which button to press next, let's an...",0,0,...,1,1,10,8,76,purple,0,0.010417,0.052083,2
296,96,purple,4,0,52,You are a bandit algorithm in a room with 5 bu...,So far you have played 96 times with your past...,"To decide which button to press next, let's an...",0,0,...,1,1,10,8,77,purple,0,0.010309,0.051546,2
297,97,purple,4,1,53,You are a bandit algorithm in a room with 5 bu...,So far you have played 97 times with your past...,"To decide which button to press next, let's an...",0,0,...,1,1,10,8,78,purple,0,0.010204,0.051020,2
298,98,purple,4,0,53,You are a bandit algorithm in a room with 5 bu...,So far you have played 98 times with your past...,"To decide which button to press next, let's an...",0,0,...,1,1,10,8,79,purple,0,0.010101,0.050505,2


In [21]:
df["SuffFail"] = 0

T = df["trial"].max()
seen_best_arm = False

for t in reversed(range(T + 1)):
    if df.loc[t, "arm_name"] == df.loc[t, "best_arm"]:
        #print(f"Chosen arm was best arm at t = {t} ")
        seen_best_arm = True  # The best arm was chosen at least once
    df.loc[t, "SuffFail"] = int(not seen_best_arm)  # 1 if best_arm was never chosen from t to T

print(df["SuffFail"])

0     0
1     0
2     0
3     0
4     0
     ..
95    0
96    0
97    0
98    0
99    0
Name: SuffFail, Length: 100, dtype: int64


In [25]:
print("For this replicate, SuffFail(T/2) = ", df.loc[df["trial"] == 50, "SuffFail"])

For this replicate, SuffFail(T/2) =  50    0
Name: SuffFail, dtype: int64


In [38]:
## CALCULATION FOR DF_COMBINED
# Sort
df_combined = df_combined.sort_values(["replicate", "trial"], ascending=[True, False])

# Identify T (max trial number)
T = df_combined["trial"].max()
T_half = T // 2  # T/2
print(T_half) #should be 49 = index of the 50th run

# Step 1: Check if the best arm was ever chosen from t to T
df_combined["seen_best_arm"] = (
    df_combined.groupby("replicate", group_keys=False)
    .apply(lambda g: g["arm_name"].eq(g["best_arm"]).iloc[::-1].cumsum().iloc[::-1] > 0)
)

# Step 2: Compute SuffFail
df_combined["SuffFail"] = (~df_combined["seen_best_arm"]).astype(int)

# Step 3: Compute SuffFailFreq(T/2)
suff_fail_freq_T_half = df_combined[df_combined["trial"] == T_half].groupby("replicate")["SuffFail"].mean().mean()

print("SuffFailFreq(T/2):", suff_fail_freq_T_half)

49
SuffFailFreq(T/2): 0.0


<ipython-input-38-5e4f73bd95ca>:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g["arm_name"].eq(g["best_arm"]).iloc[::-1].cumsum().iloc[::-1] > 0)


# Metric 2: K · MinFrac(T ) quantifies exploitation failures

uniform-like failures (expressed via K · MinFrac(T )) = exploitation fails = measures uniform-like failures

LLM selects arms in roughly equal proportions for the entirety of the T rounds and fails to exploit the acquired information to focus on the better arms.

For a particular experiment replicate R and round t, let fa(t, R) be the fraction of rounds in [1, t] in which a given arm a is chosen, MinFrac(t, R) := mina fa(t, R), and MinFrac(t) := mean({MinFrac(t, R) : replicates R}). Since MinFrac(t) ≤ 1/K, ∀t ∈ [T ], we always plot K · MinFrac(t), so as to rescale the range to [0, 1].
  


In [44]:
K = df_combined["arm_name"].unique() # number of arms = 5
print(K)

replica_numbs = df_combined["replicate"].unique()
print(replica_numbs) # R
replicate_cumulative_counts = {}

for r in replica_numbs:
  replicate_df = df_combined.groupby("replicate")
  replicate_cumulative_counts[r] = {a: (replicate_df["arm_name"] == a).cumsum() for a in K}
  df["MinFrac"] = df["trial"].apply(lambda t: min(cumulative_counts[a][t] / (t + 1) for a in K))
  df["KMinFrac"] = df["MinFrac"] * 5


# cumulative counts for each arm
cumulative_counts = {a: (df["arm_name"] == a).cumsum() for a in K}

# MinFrac(t, R)
df["MinFrac"] = df["trial"].apply(lambda t: min(cumulative_counts[a][t] / (t + 1) for a in K))
df["KMinFrac"] = df["MinFrac"] * 5

print(df[["trial", "KMinFrac"]])

['blue' 'green' 'red' 'yellow' 'purple']
[0 1 2]
    trial  KMinFrac
0       0  0.000000
1       1  0.000000
2       2  0.000000
3       3  0.000000
4       4  0.000000
..    ...       ...
95     95  0.052083
96     96  0.051546
97     97  0.051020
98     98  0.050505
99     99  0.050000

[100 rows x 2 columns]


In [55]:
K = df_combined["arm_name"].unique() # number of arms = 5
print(K)

replica_numbs = df_combined["replicate"].unique()
print(replica_numbs) # R
replicate_cumulative_counts = {}

for r in replica_numbs:
    mask = df_combined["replicate"] == r
    for a in K:
        df_combined.loc[mask, f"cumulative_count_{a}"] = (df_combined.loc[mask, "arm_name"] == a).cumsum()

    # Compute MinFrac for this replicate
    df_combined.loc[mask, "MinFrac"] = df_combined.loc[mask, "trial"].apply(
        lambda t: min(df_combined.loc[mask & (df_combined["trial"] == t), [f"cumulative_count_{a}" for a in K]].min(axis=1) / (t + 1))
    )

    # Compute KMinFrac
    df_combined.loc[mask, "KMinFrac"] = df_combined.loc[mask, "MinFrac"] * len(K)

print(df_combined[["trial", "KMinFrac"]])

['blue' 'green' 'red' 'yellow' 'purple']
[0 1 2]
     trial  KMinFrac
0        0  0.000000
1        1  0.000000
2        2  0.000000
3        3  0.000000
4        4  0.000000
..     ...       ...
295     95  0.052083
296     96  0.051546
297     97  0.051020
298     98  0.050505
299     99  0.050000

[300 rows x 2 columns]


In [56]:
# Compute MinFrac(t) as the mean MinFrac(t, R) over all replicates R
MinFrac_t = df_combined.groupby("trial")["MinFrac"].mean().reset_index(name="MinFrac_t")

# Display the result
print(MinFrac_t)

    trial  MinFrac_t
0       0   0.000000
1       1   0.000000
2       2   0.000000
3       3   0.000000
4       4   0.000000
..    ...        ...
95     95   0.010417
96     96   0.010309
97     97   0.010204
98     98   0.010101
99     99   0.010000

[100 rows x 2 columns]


# Metric 3: MedianReward

(Note: if Reward is None, filter it out, as it reports system failures)

MedianReward  = the rescaled median (over replicates) of the time-averaged total reward.

More precisely, let Φ(R) be the time-averaged total reward for a given replicate R. Then E [Φ(R)] ranges in the interval [1/2 − ∆/2, 1/2 + ∆/2]. We rescale Φ(R), by translating and multiplying, so that E [Φ(R)] ranges in [0, 1].


In [41]:
T = 100
# Compute Time-Averaged Total Reward per Replicate
df_filtered = df_combined.dropna(subset=["reward"])  # Ignore system failures
time_avg_reward_per_r = df_filtered.groupby("replicate")["reward"].sum() / T

# Compute Median of Time-Averaged Rewards
median_reward = time_avg_reward_per_r.median()

# Rescale the Median Reward
Delta = 0.2  # In our setting of the "hard" instance with K=5, then ∆=0.2
min_val = (1/2) - (Delta/2)
rescaled_median_reward = (median_reward - min_val) / Delta

print(f"Rescaled MedianReward: {rescaled_median_reward}")

Rescaled MedianReward: 0.65


# Metric 4: GreedyFrac

The fraction of greedy rounds, averaged over the replicates. A greedy round is one in which an arm with a largest average reward is selected. This is one way to quantify the extent to which a configuration behaves like Greedy.


In [42]:
df_filtered = df_combined.dropna(subset=["reward"])

# Step 2: Compute cumulative reward & count per arm per replicate
df_filtered["cumulative_reward"] = df_filtered.groupby(["replicate", "arm_name"])["reward"].cumsum()
df_filtered["cumulative_count"] = df_filtered.groupby(["replicate", "arm_name"]).cumcount() + 1

# Step 3: Compute empirical mean reward for each arm (avoid division by zero)
df_filtered["mean_reward"] = df_filtered["cumulative_reward"] / df_filtered["cumulative_count"]

# Step 4: Find the arm with the highest mean reward at each time step
max_reward_per_trial = df_filtered.groupby(["replicate", "trial"])["mean_reward"].transform("max")

# Step 5: Check if the chosen arm was greedy (has the max reward)
df_filtered["greedy_round"] = (df_filtered["mean_reward"] == max_reward_per_trial).astype(int)

# Step 6: Compute GreedyFrac per replicate and then average
greedy_frac_per_replicate = df_filtered.groupby("replicate")["greedy_round"].mean()
greedy_frac = greedy_frac_per_replicate.mean()

print(f"GreedyFrac: {greedy_frac}")


GreedyFrac: 1.0
